In [1]:
from music21 import converter, instrument, note, chord, instrument, note, chord, stream
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.utils import np_utils
from collections import Counter
from fractions import Fraction
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pathlib
import sklearn
import json
import time
import os
import gc

parse_MIDI.py

In [2]:
def load_midi_file(file_name):

    midi_sample = None
    try:
        midi_sample = converter.parse(file_name)
    except OSError as e:
        print('\nERROR loading MIDI file in load_midi_file()')
        print(e)
        quit()
    return midi_sample


def parse_midi_file(folder_path):

    notes_piano = []
    metadata_piano = {
        "note_count": 0,
        "chord_count": 0,
        "rest": 0,
        "else_count": 0
    }

    all_names = []

    for oneFile in os.listdir(folder_path):
        if oneFile.endswith(".mid"):
            all_names.append(oneFile)
            midi_file = folder_path+oneFile
            midi_sample = load_midi_file(midi_file)
            instruments = instrument.partitionByInstrument(midi_sample)

            for part in instruments.parts:

                if 'Piano' in str(part):
                    print('parsing PIANO', part)

                    notes_to_parse = part.recurse()
                    last_offset = 0

                    # note  -> n_pitch_quarterLength_deltaOffset
                    # chord -> c_pitch_quarterLength_deltaOffset
                    # rest  -> r_quarterLength_deltaOffset

                    elems_count = 0
                    for element in notes_to_parse:
                        elems_count += 1
                        if isinstance(element, note.Note):
                            delta_offset = Fraction(element.offset) - last_offset
                            last_offset = Fraction(element.offset)

                            notes_piano.append('n_'+str(element.pitch)+'_'+str(element.duration.quarterLength)+'_'+str(delta_offset))
                            metadata_piano["note_count"] += 1

                        elif isinstance(element, chord.Chord):
                            delta_offset = Fraction(element.offset) - last_offset
                            last_offset = Fraction(element.offset)

                            chord_ = '.'.join(str(n) for n in element.pitches)
                            notes_piano.append('c_' + chord_ + '_' + str(element.duration.quarterLength)+'_'+str(delta_offset))
                            metadata_piano["chord_count"] += 1

                        elif isinstance(element, note.Rest):
                            delta_offset = Fraction(element.offset) - last_offset
                            last_offset = Fraction(element.offset)

                            notes_piano.append('r_' + str(element.duration.quarterLength)+'_'+str(delta_offset))
                            metadata_piano["rest"] += 1
                        else:
                            metadata_piano["else_count"] += 1
                    print('^elems_count:', elems_count)
    print('MIDI dataset:\n', all_names)
    return notes_piano, metadata_piano


def average_f(lst):
    return sum(lst) / len(lst)


def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]


def cut_notes(uncut_notes, metadata, cuts):
    notes = uncut_notes
    stop_flag = False

    for i in range(cuts):
        count_num = Counter(notes)
        Recurrence = list(count_num.values())

        pitchnames = set(notes)
        note_to_int_before = dict((note_var, number) for number, note_var in enumerate(pitchnames))
        avg_r = round(average_f(Recurrence), 2)

        print('\nnumber of notes before:', len(notes))
        print('number of unique notes before:', len(note_to_int_before))
        print("average recurrence for a note in notes:", avg_r)
        print("most frequent note in notes appeared:", max(Recurrence), "times")
        print("least frequent note in notes appeared:", min(Recurrence), "time/s")

        # if average recurrence is more then a 100, @param avg_r is set to 100 and this will be final eliminating
        if round(avg_r) >= 100:
            avg_r = 100
            stop_flag = True

        # getting a list of elements that appear less then avarage element does
        rare_note = []
        cn_items = count_num.items()
        for index, (key, value) in enumerate(cn_items):
            if value < round(avg_r):
                m = key
                rare_note.append(m)
        print(f'number of notes occuring less than {round(avg_r)} times:', len(rare_note))

        # eleminating those elements
        for element in notes:
            if element in rare_note:
                len_before = len(notes)
                notes = remove_values_from_list(notes, element)
                elements_removed = len_before - len(notes)
                if element[:1] == 'n':
                    metadata['note_count'] -= elements_removed
                elif element[:1] == 'c':
                    metadata['chord_count'] -= elements_removed
                elif element[:1] == 'r':
                    metadata['rest'] -= elements_removed

        print("length of notes after the elemination :", len(notes))
        if stop_flag:
            break

    count_num = Counter(notes)
    Recurrence = list(count_num.values())

    avg_r = round(average_f(Recurrence), 2)

    print("\naverage recurrence for a note in notes:", avg_r)
    print("most frequent note in notes appeared:", max(Recurrence), "times")
    print("least frequent note in notes appeared:", min(Recurrence), "time/s")

    del count_num, Recurrence, pitchnames, note_to_int_before, rare_note
    gc.collect()

    return notes, metadata


def mapping(uncut_notes, metadata, sequence_len, cuts):

    if cuts > 0:
        notes, new_metadata = cut_notes(uncut_notes, metadata, cuts)
    else:
        notes, new_metadata = uncut_notes, metadata

    sequence_length = sequence_len
    pitchnames = set(notes)
    note_to_int = dict((note_var, number) for number, note_var in enumerate(pitchnames))

    nn_input = []
    nn_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]

        nn_input.append([note_to_int[char] for char in sequence_in])
        nn_output.append(note_to_int[sequence_out])

    input_count = len(nn_input)
    mapped_n_count = float(len(note_to_int))

    # reshape the input into a format compatible with LSTM layers
    nn_input = np.reshape(nn_input, (input_count, sequence_length, 1))
    # normalize input and values for mapped notes
    nn_input = nn_input / mapped_n_count
    # for i in note_to_int:
    #     note_to_int[i] = note_to_int[i] / mapped_n_count

    nn_output = np_utils.to_categorical(nn_output)

    # print metadata about notes after removing
    info_print_out(new_metadata, len(note_to_int))

    return nn_input, nn_output, note_to_int, pitchnames


def info_print_out(metadata, unique_elements_count):
    all_count = metadata['note_count'] + metadata['chord_count'] + metadata['rest']
    print('\n')
    print('_________________________________________________')
    print('\n')
    print('number of all elements:      ', all_count)
    print('notes:    ', metadata['note_count'])
    print('chords:   ', metadata['chord_count'])
    print('rests:    ', metadata['rest'])
    print('number of unique elements:   ', unique_elements_count)
    print('\n')
    print('_________________________________________________')
    print('\n')


def parse_MIDI_init(folder_path, sequence_length, cuts):
    notes_and_chords, metadata_p = parse_midi_file(folder_path)                                                         # parse MIDI file
    lstm_input, lstm_output, notes_to_int, pitch_names = mapping(notes_and_chords, metadata_p, sequence_length, cuts)   # mapping MIDI file parts

    lstm_input_shuffled, lstm_output_shuffled = sklearn.utils.shuffle(lstm_input, lstm_output)                          # shuffling input and output simultaneously
    pitch_names_len = len(pitch_names)

    del notes_and_chords
    del metadata_p
    del lstm_input
    del lstm_output
    del pitch_names
    gc.collect()

    return lstm_input_shuffled, lstm_output_shuffled, notes_to_int, pitch_names_len


train_model.py

In [3]:
def create_lstm_model(nn_input, n_pitch):
    lstm_model = Sequential()
    lstm_model.add(LSTM(
        256,
        input_shape=(nn_input.shape[1], nn_input.shape[2]),
        return_sequences=True,
    ))
    lstm_model.add(Dropout(0.6))
    lstm_model.add(LSTM(256, return_sequences=True))
    lstm_model.add(Dropout(0.6))
    lstm_model.add(LSTM(256))
    lstm_model.add(Dropout(0.6))

    lstm_model.add(Dense(256))
    lstm_model.add(Dropout(0.6))

    lstm_model.add(Dense(n_pitch))
    lstm_model.add(Activation('sigmoid'))
    lstm_model.compile(optimizer='Adam', loss='categorical_crossentropy')

    return lstm_model


def load_weight_to_model(empt_model, weight):
    filepath = f'weights\\toLoadWeights\\{weight}.hdf5'
    try:
        empt_model.load_weights(filepath)
    except OSError as e:
        print('\nERROR loading weights file in load_weight_to_model()')
        print(e)
        quit()
    return empt_model


def train_lstm(nn, nn_input, nn_output, epochs, batch_size):

    filepath = "weights\\weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath, monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    # print('Input shape: ' + nn_input.shape)
    # print('Output shape: ' + nn_output.shape)
    # print('X[0]: ', nn_input[0])
    # print('argmax y[0]: ', np.argmax(nn_output[0]))
    # print('argmax y[0] / 182: ', np.argmax(nn_output[0]) / float(182))

    # gpus = tf.config.experimental.list_physical_devices('GPU')
    # if gpus:
    #     try:
    #         # Currently, memory growth needs to be the same across GPUs
    #         for gpu in gpus:
    #             tf.config.experimental.set_memory_growth(gpu, True)
    #         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    #         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    #         data = nn.fit(nn_input, nn_output, epochs=250, batch_size=16, callbacks=callbacks_list)  # batch_size=64
    #         return nn, data
    #     except RuntimeError as e:
    #         # Memory growth must be set before GPUs have been initialized
    #         print(e)

    # with tf.device('/GPU:0'):
    data = nn.fit(nn_input, nn_output, epochs=epochs, batch_size=batch_size, callbacks=callbacks_list)     # batch_size=64

    fig = plt.figure()
    ax = plt.subplot(111)
    plt.plot(data.history['loss'], label=f'Loss hodnota', lw=2)
    plt.title('Trénovanie celého datasetu')
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                     box.width, box.height * 0.9])
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.135),
              fancybox=True, shadow=True, ncol=5)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.draw()
    plt.show()
    fig.savefig('tests\\cely-dataset-bs64_nieco.pdf')
    plt.clf()
    return nn


def train_model_init(lstm_input, lstm_output, pitch_names_len, epochs, batch_size, model_training):

    empty_model = create_lstm_model(lstm_input, pitch_names_len)                        # load layers of NN to model

    if model_training["bool"]:
        model = train_lstm(empty_model, lstm_input, lstm_output, epochs, batch_size)    # train NN
    else:
        model = load_weight_to_model(empty_model, model_training["weight"])             # load weights to model

    return model


generate_music.py

In [4]:
# source of this function : https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac


def create_midi_file(output, mapping_keys, length, index, new_file_name):

    unmapped_from_int = []
    converted = []
    notes = []
    offset = 0

    metadata = {
        "note": 0,
        "chord": 0,
        "rest": 0
    }

    # # unmapping notes, chores and rests from output integers
    for element in output:
        for key in mapping_keys:
            if int(mapping_keys.get(key)) == element:
                unmapped_from_int.append(key)
                break

    # creating note, chores and rest objects
    for element in unmapped_from_int:
        if 'n_' in element:                         # note
            element = element[2:]                                                       # cut the 'n_' mark
            offset += Fraction(element.split('_')[2])
            note_ = note.Note(element.split('_')[0])                                    # creating note
            note_.duration.quarterLength = convert_to_float(element.split('_')[1])      # adding duration quarterLength
            note_.offset = offset                                                       # adding offset
            note_.storedInstrument = instrument.Piano()
            converted.append(note_)                                                     # appending final array
            metadata['note'] = metadata['note'] + 1

        elif 'c_' in element:                       # chord
            element = element[2:]                                                       # cut the 'c_' mark
            offset += Fraction(element.split('_')[2])
            notes_in_chord = element.split('_')[0].split('.')                           # gettig notes as string from element

            notes.clear()
            for current_note in notes_in_chord:
                new_note = note.Note(current_note)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)

            chord_ = chord.Chord(notes)                                                 # creating chord form notes
            chord_.duration.quarterLength = convert_to_float(element.split('_')[1])     # adding duration quarterLength
            chord_.offset = offset                                                      # adding offset
            converted.append(chord_)                                                    # appending final array
            metadata['chord'] = metadata['chord'] + 1

        elif 'r_' in element:                       # rest
            element = element[2:]                                                       # cut the 'r_' mark
            offset += Fraction(element.split('_')[1])
            rest_ = note.Rest()                                                         # creating rest
            rest_.duration.quarterLength = convert_to_float(element.split('_')[0])      # adding duration quarterLength
            rest_.offset = offset                                                       # adding offset
            rest_.storedInstrument = instrument.Piano()
            converted.append(rest_)                                                     # appending final array
            metadata['rest'] = metadata['rest'] + 1

    print(f'\nElements of newly generated music with index={index}: ', metadata)

    try:
        midi_stream = stream.Stream(converted)
        midi_stream.write('midi', fp='midi_samples\\outputs\\' + f'{new_file_name}' + str(index) + '.mid') 
#         midi_stream.write('midi', fp=f'midi_samples\outputs\{new_file_name}{str(index)}.mid')
        print('Created new MIDI file')
        midi_stream.show('midi')
    except OSError as e:
        print('\nERROR creating MIDI file in create_midi_file()')
        print(e)


def generate_music(nn_model, nn_input, mapped_notes, length):

    generated_music = []
    sequence_len = len(nn_input[0])
    mapped_notes_count = len(mapped_notes)
    start = np.random.randint(0, len(nn_input) - 1)
    # int_to_notes = {v: k for k, v in mapped_notes.items()}
    pattern = nn_input[start]

    note_input = np.array(pattern).reshape((1, sequence_len, 1))
    note_input = note_input / float(mapped_notes_count)
    # note_input = note_inputQ / float(mapped_notes_count)

    for new_note in range(length):
        note_output = nn_model.predict(note_input, verbose=0)
        note_output_max = np.argmax(note_output)
        generated_music.append(note_output_max)

        pattern = np.append(pattern, note_output_max / float(mapped_notes_count))
        pattern = pattern[1:sequence_len + 1]

        note_input = np.array(pattern).reshape((1, sequence_len, 1))

    return generated_music


def generate_music_init(model, lstm_input, notes_to_int, length, index, new_file_name):
    new_music = generate_music(model, lstm_input, notes_to_int, length)         # predict new music
    create_midi_file(new_music, notes_to_int, length, index, new_file_name)     # save new music to MIDI file


main.py

In [ ]:
try:
    start_time = time.time()
    print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

    with open('config.json', 'r') as f:
        config = json.load(f)

    sequence_length = int(config["sequence_length"])
    midi_files_folder = 'midi_samples\\' + config["dataset_folder"] + '\\'
    cuts = int(config["cuts"])
    epochs = int(config["epochs"])
    batch_size = int(config["batch_size"])
    new_music_length = int(config["new_music_length"])
    tracks_to_generate = int(config["tracks_to_generate"])
    new_music_file_name = config["new_music_file_name"]
    model_training = config["training"]

    lstm_input, lstm_output, notes_to_int, pitch_names_len = parse_MIDI_init(midi_files_folder, sequence_length, cuts)
    model = train_model_init(lstm_input, lstm_output, pitch_names_len, epochs, batch_size, model_training)

    for i in range(tracks_to_generate):
        generate_music_init(model, lstm_input, notes_to_int, new_music_length, i, new_music_file_name)

    # generate_music.init(model, lstm_input, notes_to_int, 100)
    # generate_music.init(model, lstm_input, notes_to_int, 150)
    # generate_music.init(model, lstm_input, notes_to_int, 200)
    # generate_music.init(model, lstm_input, notes_to_int, 250)
    # generate_music.init(model, lstm_input, notes_to_int, 300)

    end_time = time.time()
    print('Time:', int(end_time - start_time), 's')
    print('\n--- END ---\n')
    # model.summary()

except OSError as e:
    print('\nERROR loading config file in main.py')
    print(e)
    quit()


Num GPUs Available:  1
parsing PIANO <music21.stream.Part Piano right>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano>
^elems_count: 6876
parsing PIANO <music21.stream.Part Piano left>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano right>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano>
^elems_count: 6876
parsing PIANO <music21.stream.Part Piano left>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano right>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano>
^elems_count: 6876
parsing PIANO <music21.stream.Part Piano left>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano right>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano>
^elems_count: 6876
parsing PIANO <music21.stream.Part Piano left>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano right>
^elems_count: 5
parsing PIANO <music21.stream.Part Piano>
^elems_count: 6876
parsing PIANO <music21.stream.Part Piano left>
^elems_count: 5
MIDI dataset:
 ['mz_333_3.mid', 'mz_3


Elements of newly generated music with index=1:  {'note': 71, 'chord': 5, 'rest': 24}
Created new MIDI file



Elements of newly generated music with index=2:  {'note': 86, 'chord': 1, 'rest': 13}
Created new MIDI file



Elements of newly generated music with index=3:  {'note': 91, 'chord': 0, 'rest': 9}
Created new MIDI file



Elements of newly generated music with index=4:  {'note': 71, 'chord': 2, 'rest': 27}
Created new MIDI file


...


Elements of newly generated music with index=9:  {'note': 84, 'chord': 2, 'rest': 14}
Created new MIDI file



Elements of newly generated music with index=10:  {'note': 83, 'chord': 5, 'rest': 12}
Created new MIDI file



Elements of newly generated music with index=11:  {'note': 90, 'chord': 0, 'rest': 10}
Created new MIDI file



Elements of newly generated music with index=12:  {'note': 76, 'chord': 7, 'rest': 17}
Created new MIDI file



Elements of newly generated music with index=13:  {'note': 76, 'chord': 6, 'rest': 18}
Created new MIDI file
